In [ ]:
# LP Curs 2019-20, Q2

# Sessió 2 Python: Memorització, Clausures i Decoradors


#### Cal tenir instal·lat el mòdul 'ttictoc'

In [12]:
!pip install ttictoc

Defaulting to user installation because normal site-packages is not writeable


#### Necessitarem la funció 'test' en aquesta sessió. Servirà per calcular el temps que triga l'execució d'una crida a una funció. No és l'única manera de fer això en Python...



In [40]:
from ttictoc import tic, toc

def test(f, n):
    tic();
    valor = f(n)
    elapsed = toc();
    print('valor:', valor)
    print('temps(s):', '{:.5f}'.format(elapsed))

## Memorització amb valors per defecte

In [ ]:
def fib(n):
    if n in [0, 1]:
        return n
    return fib(n-1) + fib(n-2)

test(fib,40)

In [45]:
def efib (n, mem={0:0, 1:1}):
    if n not in mem:
        mem[n] = efib(n-1) + efib(n-2)
        print("Acabo d'assignar un valor a mem[", n, "] = ", mem[n])
    return mem[n]

In [46]:
test(efib,10)

Acabo d'assignar un valor a mem[ 2 ] =  1
Acabo d'assignar un valor a mem[ 3 ] =  2
Acabo d'assignar un valor a mem[ 4 ] =  3
Acabo d'assignar un valor a mem[ 5 ] =  5
Acabo d'assignar un valor a mem[ 6 ] =  8
Acabo d'assignar un valor a mem[ 7 ] =  13
Acabo d'assignar un valor a mem[ 8 ] =  21
Acabo d'assignar un valor a mem[ 9 ] =  34
Acabo d'assignar un valor a mem[ 10 ] =  55
valor: 55
temps(s): 0.00775


In [47]:
def efib (n, mem={0:0, 1:1}):
    if n not in mem:
        mem[n] = efib(n-1) + efib(n-2)
    return mem[n]

In [48]:
test(efib,40)

valor: 102334155
temps(s): 0.00006


## Memorització amb funcions niuades

In [49]:
def efib2(x):
    mem = {0:0, 1:1} # la memòria
    
    def mfib(n):
        if n not in mem: 
            mem[n] = mfib(n-1) + mfib(n-2)
        return mem[n]
    
    return mfib(x)

In [51]:
test(efib2,40)

valor: 102334155
temps(s): 0.00007


## Clausures (*Closures*)

#### Versió de la funció 'test' utlitzant una closure. L'import del ttictoc ja l'hem fet abans, per això no ens cal aquí...



In [52]:
def test(n):
    
    def prec(x):
        return '{:.5f}'.format(x)
    
    def clausura(f):
        tic();
        valor = f(n)
        elapsed = toc();
        print('valor:', valor)
        print('temps(s):', prec(elapsed))
    
    return clausura


#### Fixeu-vos que la funció 'clausura' retornada fa servir el paràmetre 'n' i la funció 'prec'. Totes dues variables (considerem el nom d'una funció com a variable també) han estat CAPTURADES en el moment de CREAR 'clausura'. Quan aquesta funció sigui retornada, les variables 'n' i 'prec' ja no existiran, ja que eren locals a 'test'.
#### Tot i així, quan utilitzem la funció retornada FORA de l'abast de les variables 'n' i 'prec', aquestes variables continuaran existint "*dins*" de la funció retornada.


In [ ]:
test30 = test(30)


#### En aquest punt 'test' ja s'ha executat i les variables 'n' i 'prec', com a variables locals de la funció 'test', ja han desaparegut... però no del tot. Quan executem 'test30' aquestes variables, aquest *ESTAT*, és utilitzat per la funció per fer la seva feina. Les variables han estat **CAPTURADES** en crear la funció retornada per 'test'.


In [0]:
test30(fib)

In [0]:
import math
test30(math.factorial)

#### Veiem un exemple una mica més "extrem" d'aquest fenòmen. Definim una estructura de dades 'Pila' *SENSE* utilitzar classes, només closures. Aprofitarem aquesta propietat de capturar el context lèxic per aconseguir encapsulament, és a dir, l'estat capturat és inaccessible des de l'exterior, només es pot consultar/alterar mitjançant les closures que l'han capturat.



In [0]:
from collections import namedtuple
Pila = namedtuple('Pila', 'push pop empty top size') 

def creador_piles():
    pila = []
    num_elems = 0

    def push(x):
        nonlocal num_elems
        pila.append(x)
        num_elems += 1

    def pop():
        # pila no es buida
        nonlocal num_elems
        x = pila.pop()
        num_elems -= 1
        return x

    def empty():
        return num_elems == 0

    def top():
        # pila no es buida 
        return pila[num_elems - 1]

    def size():
        return num_elems
    
    return Pila(push,pop,empty,top,size)

In [0]:
p = creador_piles()
p.push(3)
p.push(5)
p.push("Hola")
p.push("Món")
print(p.size())
print(p.pop())
print(p.pop())
print(p.pop())
print(p.pop())
print(p.empty())

#### Fixeu-vos que ens hem vist obligats a dir, *dins de les funcions que alteren la variable 'num_elems' via assignacions*, que aquesta variable no és local (i per tant, és la variable del context lèxic capturada). Penseu... **en una funció local, amb un cert context lèxic que capturar, com sabem si una variable és local o no? Python decideix que una variable és local en el moment que algún valor li és assignat**. Per això cal fer explícit que aquesta variable no és local. Fixeu-vos també que a la variable 'pila' no se li assigna res en cap moment, per tant no cal dir que no és local, Python ja treballa sobre la variable capturada. 

## Memorització genèrica

In [53]:
def memoritza (f):
                   # Això es pot fer perquè els diccionaris són mutables.
    mem = {}       # mem representa la memòria
    
    def f2 (x):    # f2 captura 'f' i 'mem'
        if x not in mem:
            mem[x] = f(x)
        return mem[x]
    
    return f2

def fib(n):
    if n in [0, 1]:
        return n
    return fib(n-1) + fib(n-2)

In [54]:
test40 = test(40)
fib = memoritza(fib)   # Com que la funció és recursiva hem de redefinir la funció
test40(fib)

valor: 102334155
temps(s): 0.00012


## Decoradors

#### Imaginem que volem fer alguna cosa abans i/o després de cridar alguna funció (logging, tracing, etc). Com que en Python tenim funcions/closures d'ordre superior no és massa difícil d'imaginar la manera de fer-ho...

In [56]:
def testDec(f):
    def wrapper(*args):
        valor = f(*args)
        print('f(' + str(args[0]) + ') = ' + str(valor))
        return valor
    return wrapper

def efib (n, mem={0:0,1:1}):
    if n not in mem:
        mem[n] = efib (n-1) + efib (n-2)
    return mem[n]

efib = testDec(efib)

In [57]:
test4 = test(4)
test4(efib)

f(1) = 1
f(0) = 0
f(2) = 1
f(1) = 1
f(3) = 2
f(2) = 1
f(4) = 3
valor: 3
temps(s): 0.00066


#### Tenim una notació alternativa (syntactic sugar) per a aquests casos:

#### Si hem definit:

#### `def g(ff): . . . `   on `ff` és una funció, i `g` ha de retornar una funció

#### Aleshores, escriure:

#### `def f (...): . . .`
#### `f = g(f)`

#### és el mateix que dir:

#### `@g`
#### `def f (...): . . .`




In [58]:
@testDec
def efib2 (n, mem={0:0,1:1}):    # efib2 és idèntica a efib, però li canviem el nom per evitar confusions
    if n not in mem:
        mem[n] = efib2 (n-1) + efib2 (n-2)
    return mem[n]

In [0]:
test4 = test(4)
test4(efib2)

#### Podem afegir arguments parametritzant els decoradors, però al final cal retornar una funció que tingui una funció com a paràmetre...

In [59]:
def testInterval(inici, fi):
    
    def decorador(f):
        
        def wrapper(*args):
            valor = f(*args)
            n = args[0]
            if inici <= n <= fi:
                print('f(' + str(n) + ') = ' + str(valor))
            return valor
        
        return wrapper
    
    return decorador

def efib (n, mem={0:0,1:1}):
    if n not in mem:
        mem[n] = efib (n-1) + efib (n-2)
    return mem[n]

efib = testInterval(35,40)(efib)  # recordem que testInterval(35,40) retorna una funció

In [60]:
test40 = test(40)
test40(efib)

f(35) = 9227465
f(36) = 14930352
f(35) = 9227465
f(37) = 24157817
f(36) = 14930352
f(38) = 39088169
f(37) = 24157817
f(39) = 63245986
f(38) = 39088169
f(40) = 102334155
valor: 102334155
temps(s): 0.00058


#### **Exercici**: Per què surten aquests números i en aquest ordre?
#### Em refereixo a fib(35), fib(36), fib(35),... alguns repetits, alguns no...

In [0]:
# Un cop més, podem escriure...

@testInterval(35, 40)
def efib2 (n, mem={0:0,1:1}):
    if n not in mem:
        mem[n] = efib2 (n-1) + efib2 (n-2)
    return mem[n]

In [0]:
test40 = test(40)
test40(efib2)

## Memorització genèrica amb decoradors

In [0]:
def memoritza (f):
    mem = {}
    
    def f2 (x):
        if x not in mem:
            mem[x] = f(x)
        return mem[x]
    
    return f2

@memoritza
def fib(n):
    if n in [0, 1]:
        return n
    return fib(n-1) + fib(n-2)

In [0]:
fib(40)